In [1]:
#!pip install simple_salesforce #allows us to work with salesforce API

In [2]:
import os
orig_working_directory=os.getcwd()
orig_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW\\ETL_Project'

In [3]:
os.chdir(os.path.join('..'))
curr_working_directory=os.getcwd()
curr_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW'

## Import Dependencies

In [4]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

## Import configuration variables

In [5]:
from salesforce_config import sf_username, sf_password, sf_security_token
from salesforce_config import remote_db_endpoint, remote_db_port
from salesforce_config import remote_db_name, remote_db_user, remote_db_pwd

## Set up simple_salesforce

In [6]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

## Connect to your MySQL database

In [7]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Query Data

In [8]:
student_data = pd.read_sql("SELECT * FROM student", conn)
student_data.head(5)

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


## Transform Data

In [9]:
student_data['FullName'] = student_data['FirstName'].str.cat(student_data['LastName'], sep=" ")
student_data=student_data.rename(columns={"ID_Student":"ID_Student__c","StudentID":"StudentID__c","LastName":"LastName__c",
                                       "FirstName":"FirstName__c","MiddleName":"MiddleName__c","Gender":"Gender__c",
                                          "FullName":"Name"})
student_data=student_data[["ID_Student__c","StudentID__c","LastName__c","FirstName__c","MiddleName__c","Gender__c","Name"]]
student_data.head(5)

,ID_Student__c,StudentID__c,LastName__c,Name,MiddleName__c,Gender__c
0,33,25004961,Moore,Heather,Alice,F
1,34,25003514,Multak,Ilana,Cecille,F
2,35,25005833,Murillo,Jessica,Dorothy,F
3,36,25002589,Romanowski,Kandra,Genevieve,F
4,37,25007185,Hoffer,Katherine,Lynnette,F


## Convert DataFrame to a list of Dictionaries

In [10]:
student_load = student_data.to_dict('records')
student_load

[{'ID_Student__c': 33,
  'StudentID__c': '25004961',
  'LastName__c': 'Moore',
  'Name': 'Heather',
  'MiddleName__c': 'Alice',
  'Gender__c': 'F'},
 {'ID_Student__c': 34,
  'StudentID__c': '25003514',
  'LastName__c': 'Multak',
  'Name': 'Ilana',
  'MiddleName__c': 'Cecille',
  'Gender__c': 'F'},
 {'ID_Student__c': 35,
  'StudentID__c': '25005833',
  'LastName__c': 'Murillo',
  'Name': 'Jessica',
  'MiddleName__c': 'Dorothy',
  'Gender__c': 'F'},
 {'ID_Student__c': 36,
  'StudentID__c': '25002589',
  'LastName__c': 'Romanowski',
  'Name': 'Kandra',
  'MiddleName__c': 'Genevieve',
  'Gender__c': 'F'},
 {'ID_Student__c': 37,
  'StudentID__c': '25007185',
  'LastName__c': 'Hoffer',
  'Name': 'Katherine',
  'MiddleName__c': 'Lynnette',
  'Gender__c': 'F'},
 {'ID_Student__c': 38,
  'StudentID__c': '25006014',
  'LastName__c': 'Poocharoen',
  'Name': 'Pariya',
  'MiddleName__c': 'Mariette',
  'Gender__c': 'F'},
 {'ID_Student__c': 39,
  'StudentID__c': '25007528',
  'LastName__c': 'Mantrala'

## Inserting rows in bulk

In [11]:
try:
    sf.bulk.Student__c.insert(student_load) #Student__c is the API name
except Exception as e:
    print(e)